## Dataset Creation

In [ ]:
import random as rd
import pandas as pd

def create_dataset(path = "/Users/arunkaul/Desktop/Random.txt"): #Add The Path Of Text File That Contains All The Text On Which You will Train Your Model
    n = 5
    dataset_book = open(path,"r")
    
    list_of_lines = dataset_book.readlines()
    list_of_lines_data = []
    list_data = []
    # Cleaning Data
    for line in list_of_lines:
        list_of_lines_data.append(line.replace("?","").replace(",","").replace(".","").replace("!","").replace("\n","").replace("<s>","").replace("</s>","").replace("'","").strip())
    
    list_of_lines_data = list_of_lines_data[1:len(list_of_lines_data)-1]

    #Adding The Text Data To New List
    for i in list_of_lines_data:
        if len(i) <= 6:
            list_of_lines_data.remove(i)	

    print(list_of_lines_data)
    for i in list_of_lines_data:
        i = i.split(" ")
        for a in i:
            list_data.append(a)
    
    lines = []
    word = []
    lines_data = []
    index = []

    #Converting that List Into Data Of 2 word Lines As Input And 1 Word As Output
    
    for i in range(0,len(list_data)-n,n):
        lines.append(list_data[i:i+n])
        word.append(list_data[i+n])
        
    c = 0
    for i in lines:
        c+=1
        lines_data.append(i[0]+" "+ i[1]+" "+i[2]+" "+i[3]+" " + i[4])
        index.append(c)  
        
    # Creating A Pandas Dataframe For Further Use
    dataset = {'lines':lines_data,'word':word}
    dataset_final = pd.DataFrame(dataset)
    dataset_final["lines"] = dataset_final["lines"].apply(lambda x: str(x).lower())
    dataset_final.to_csv("File.csv")
    return dataset_final

dataset_books = create_dataset()
dataset_books.tail(1000)

## Data Preprocessing

In [ ]:
import keras
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import Tokenizer
from sklearn.model_selection import train_test_split
import tensorflow

features = 200000


tokenizer_y = Tokenizer(features)

#Fitting The Values To Words
tokenizer_y.fit_on_texts(dataset_books['word'].values)

#Converting Words To NumPy Array
Y = tokenizer_y.texts_to_sequences(dataset_books['word'].values)

#Converting Lines To NumPy Array
X = tokenizer_y.texts_to_sequences(dataset_books['lines'].values)

# Removing Wrong Data
c = 0
D = 0
for i in X:
    if(len(i) > 5):
        c +=1
        index = X.index(i)
        Y.pop(index)
        X.remove(i)

for i in Y:
    if(len(i) > 1):
        D +=1
        index = Y.index(i)
        X.pop(index)
        Y.remove(i)
        
X = pad_sequences(X,maxlen = 5)
Y = pad_sequences(Y,maxlen = 1)

#Creating Traing Data
X_train, X_test, Y_train, Y_test = train_test_split(
    X, Y, test_size=0.15, random_state=42)
print(Y)
print(tokenizer_y.word_index)

## Model Creation

In [ ]:
import tensorflow as tf
from tensorflow.keras.optimizers import *
from keras.models import Sequential
from keras.layers import *
import matplotlib.pyplot as plt

model = Sequential()

model.add(Embedding(features,32))
model.add(GRU(600, return_sequences=False))
model.add(Flatten())
model.add(Dense(11803,activation = "softmax"))
          
print(X_train.shape)
model.build(input_shape = X_train.shape)

model.compile(
    loss="sparse_categorical_crossentropy",
    optimizer="adam",
    metrics = ["accuracy"]
)

callback = keras.callbacks.EarlyStopping(monitor='val_accuracy',
                                              patience=2)
print(model.summary())


batch_size = 32

info = model.fit(
    X_train,
    Y_train,
    epochs=30,
    batch_size=batch_size,
    shuffle=True,
    validation_data=(X_test, Y_test),
    callbacks = [callback]
)

model.save('model_6.keras')
plt.title("Loss Analysis")
plt.plot(info.history["loss"], label="Loss")
plt.plot(info.history["accuracy"], label="Accuracy")
plt.legend()
plt.show()

## Prediction

In [ ]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.optimizers import *
from tensorflow.keras.losses import *
from tensorflow.keras.layers import *
import numpy as np
from keras.models import load_model
from keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.optimizers import *

def predict_next_word(n):
    
    model = load_model('model_6.keras')
    model.compile(loss = "mean_squared_error")
    
    #Enter The Input
    x = input("Enter Sentence :")
        
    y = x.strip()
    x = x.strip()
    x = x.split(" ")
    t = x[len(x)-5]+" "+ x[len(x)-4]+" "+x[len(x)-3]+" "+x[len(x)-2] + " " + x[len(x)-1]
        
    sample = {}
    sample.update({'lines':[t]})
    sample = pd.DataFrame(sample)
        
    Text = tokenizer_y.texts_to_sequences(sample['lines'].values)
    Text = pad_sequences(Text,maxlen = 5)
    dict = tokenizer_y.word_index
    inv_map = {v: k for k, v in dict.items()}
    s = model.predict(Text)
    prediction = list(s[0])
    
    for i in range(n):
        maxi = max(prediction)
        index = prediction.index(maxi)
        final_output = y + " " + inv_map[index]
        print(final_output)
        print("Probability :",maxi)
        prediction.remove(maxi)

##Output Top 5 Best Predictions
predict_next_word(5)



## Requirements
#### pip install tensorflow==2.16.1
#### pip install keras==3.3.3
#### pip install pandas==2.1.4
#### pip install scikit-learn
#### pip install numpy==1.26.4

# Author

## Aryan Kaul